#### Домашнее задание 5

1.Попробуйте изменить параметры нейронной сети, работающей с датасетом imdb, либо нейронной сети, работающей airline-passengers (она прилагается вместе с датасетом к уроку в виде отдельного скрипта) так, чтобы улучшить её точность. Приложите анализ

2.Попробуйте изменить параметры нейронной сети, генерирующей текст таким образом, чтобы добиться генерации как можно более осмысленного текста. Пришлите лучший текст из получившихся и опишите предпринятые для его получения действия. Можно использовать текст другого произведения

In [1]:
import numpy as np

from tensorflow.keras.layers import Dense, Activation, Embedding, LSTM, GRU
#from tensorflow.keras.layers.recurrent import SimpleRNN, LSTM, GRU
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import imdb

In [2]:
def model_testing(max_features, max_len, batch_size):

    (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
    x_train = sequence.pad_sequences(x_train, maxlen=max_len)
    x_test = sequence.pad_sequences(x_test, maxlen=max_len)

    model = Sequential()
    model.add(Embedding(max_features, 128))
    model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
              optimizer='nadam', # в данном случае nadam показал результаты лучше, чем adam и adamax
              metrics=['accuracy'])
    
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=1, 
              validation_data=(x_test, y_test))
  
    score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
  
    print('Результат при тестировании:', score)
    print('Тестовая точность:', acc)

In [3]:
# сначала зафиксируем max_len и batch_size

for max_feat in [15000, 17500, 20000, 22000]:
    print(f'\nПри max_features = {max_feat}:\n')
    model_testing(max_feat, 125, 200)


При max_features = 15000:

125/125 [==============================] - 29s 233ms/step - loss: 0.3421 - accuracy: 0.8546
Результат при тестировании: 0.3420848846435547
Тестовая точность: 0.854640007019043

При max_features = 17500:

125/125 [==============================] - 45s 349ms/step - loss: 0.3592 - accuracy: 0.8502
Результат при тестировании: 0.35923776030540466
Тестовая точность: 0.8501999974250793

При max_features = 20000:

125/125 [==============================] - 56s 450ms/step - loss: 0.3451 - accuracy: 0.8468
Результат при тестировании: 0.34506916999816895
Тестовая точность: 0.8468000292778015

При max_features = 22000:

125/125 [==============================] - 55s 442ms/step - loss: 0.3431 - accuracy: 0.8586
Результат при тестировании: 0.3430933654308319
Тестовая точность: 0.8585600256919861


#### по результатам данного эксперимента видно, что при увеличении количества фич точность несущественно снижается, в а в последнем случае даже немного увеличивается.

In [4]:
# зафиксируем max_features и batch_size

for max_l in  [100, 125, 150, 175]:
    print(f'\nПри max_len = {max_l}:\n')
    model_testing(15000, max_l, 200)


При max_len = 100:

125/125 [==============================] - 17s 137ms/step - loss: 0.3758 - accuracy: 0.8396
Результат при тестировании: 0.37584468722343445
Тестовая точность: 0.8396400213241577

При max_len = 125:

125/125 [==============================] - 66s 534ms/step - loss: 0.3375 - accuracy: 0.8534
Результат при тестировании: 0.3375228941440582
Тестовая точность: 0.8533999919891357

При max_len = 150:

125/125 [==============================] - 59s 472ms/step - loss: 0.3482 - accuracy: 0.8503
Результат при тестировании: 0.3481884300708771
Тестовая точность: 0.8502799868583679

При max_len = 175:

125/125 [==============================] - 65s 517ms/step - loss: 0.4646 - accuracy: 0.8285
Результат при тестировании: 0.4645947217941284
Тестовая точность: 0.8285199999809265


#### Наилучшее значение при max_len = 125, далее точность снижается

In [5]:
# зафиксируем max_features и max_len

for batch_s in [150, 175, 200, 250]:
    print(f'\nПри batch_size = {batch_s}:', '\n', )
    model_testing(15000, 125, batch_s)


При batch_size = 150: 

167/167 [==============================] - 33s 200ms/step - loss: 0.3380 - accuracy: 0.8561
Результат при тестировании: 0.3380450904369354
Тестовая точность: 0.8561199903488159

При batch_size = 175: 

143/143 [==============================] - 21s 147ms/step - loss: 0.3298 - accuracy: 0.8585
Результат при тестировании: 0.32977473735809326
Тестовая точность: 0.8584799766540527

При batch_size = 200: 

125/125 [==============================] - 70s 558ms/step - loss: 0.3451 - accuracy: 0.8504
Результат при тестировании: 0.3450532555580139
Тестовая точность: 0.8504400253295898

При batch_size = 250: 

100/100 [==============================] - 36s 362ms/step - loss: 0.3671 - accuracy: 0.8436
Результат при тестировании: 0.3670826852321625
Тестовая точность: 0.843559980392456


##### Увеличение batch_size приводит к росту точности в интервале до 175, далее точность снижается.

##### Оптимальные параметры для модели 

- max_features = 15000
- max_len = 125
- batch_size = 175

Результат при тестировании: 0.32977473735809326

Тестовая точность: 0.8584799766540527

#### Задание 2

In [2]:
# Построчное чтение файла Alice_in_Wonderland

with open("Alice_in_Wonderland.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("ascii", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set([c for c in text])
nb_chars = len(chars)

In [3]:
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

In [4]:
SEQLEN, STEP = 20, 1
input_chars, label_chars = [], []

In [5]:
# Конвертация data в серии разных SEQLEN-length субпоследовательностей

for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])

In [6]:
# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=bool)
y = np.zeros((len(input_chars), nb_chars), dtype=bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

In [7]:
# Установка ряда метапамертров  для нейронной сети и процесса тренировки

BATCH_SIZE, HIDDEN_SIZE = 1024, 512
NUM_ITERATIONS = 10 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 5
NUM_PREDS_PER_EPOCH = 100

In [8]:
model = Sequential()
model.add(
    GRU(  
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [10]:
# Выполнение серий тренировочных и демонстрационных итераций 

for iteration in range(NUM_ITERATIONS):

    # Для каждой итерации запуск передачи данных в модель 
    print("\n")
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        print(y_pred, end="")

        test_chars = test_chars[1:] + y_pred
print()



Итерация #: 0
Epoch 1/5
156/156 [==============================] - 52s 329ms/step - loss: 1.9081
Epoch 2/5
156/156 [==============================] - 51s 325ms/step - loss: 1.7673
Epoch 3/5
156/156 [==============================] - 51s 329ms/step - loss: 1.6515
Epoch 4/5
156/156 [==============================] - 51s 324ms/step - loss: 1.5550
Epoch 5/5
156/156 [==============================] - 53s 338ms/step - loss: 1.4713
Генерация из посева: lat upon their faces
lat upon their faces in the wonder would to see would to see would to see would to see would to see would to see would t

Итерация #: 1
Epoch 1/5
156/156 [==============================] - 49s 316ms/step - loss: 1.3965
Epoch 2/5
156/156 [==============================] - 51s 327ms/step - loss: 1.3290
Epoch 3/5
156/156 [==============================] - 53s 342ms/step - loss: 1.2662
Epoch 4/5
156/156 [==============================] - 55s 351ms/step - loss: 1.2080
Epoch 5/5
156/156 [==============================] - 52s 33


    Для улучшения обучения увеличил количество итераций. Если увеличить HIDDEN_SIZE -- обучение пройдет быстрее, loss снижается. Увеличение SEQLEN так же увеличит скорость обучения модели, но эпохи становятся длительнее, так как моель обучается на более длинной последовательности.
